In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api

In [2]:
# google news corpus is used for training containing 3 billion words
# 300 dimensional vectors for 3 million words and phrases

In [3]:
word_emb = api.load('word2vec-google-news-300')

In [4]:
word_emb.get_index('car')

385

In [5]:
word_emb.get_vector('car').shape

(300,)

In [6]:
word_emb['man'];

In [7]:
word_emb.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824871301651001),
 ('teenager', 0.6586930155754089),
 ('teenage_girl', 0.6147903203964233),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.571636438369751),
 ('robber', 0.5585119128227234),
 ('Robbery_suspect', 0.5584409832954407),
 ('teen_ager', 0.5549196600914001),
 ('men', 0.5489763021469116)]

In [8]:
# so it contains vectors of words 

In [9]:
# types of word to vec => 1. CBOW   2. Skip-gram

# Custom word2vec embeddings

In [10]:
import nltk
import pandas as pd
import numpy as np
import os

In [11]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [13]:
# creating a list to keep the entire data
# the data used is game of thrones books (txt files) from kaggle
# link -> https://www.kaggle.com/datasets/khulasasndh/game-of-thrones-books
story = []

In [14]:
# list of text files

for file in os.listdir("./gotbooks"):
    print(file)

003ssb.txt
001ssb.txt
005ssb.txt
002ssb.txt
004ssb.txt


In [15]:
folder_path = "./gotbooks"

In [18]:
# function to tokenize, preprocess and store all the files in empty dictionary story

def preprocessor(folder_path_of_files):
    story= []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        with open (file_path, encoding= 'unicode_escape') as f:
            corpus = f.read()
        raw_sent = sent_tokenize(corpus)
        # preprocess using simple_preprocess and store in story list
        for sent in raw_sent:
            story.append(simple_preprocess(sent))
            
    return story

In [19]:
story = preprocessor(folder_path)

In [20]:
story[0]

['storm',
 'of',
 'swords',
 'book',
 'three',
 'of',
 'song',
 'of',
 'ice',
 'and',
 'fire',
 'by',
 'george']

In [22]:
story[3]

['with',
 'such',
 'structure',
 'the',
 'narrative',
 'cannot',
 'be',
 'strictly',
 'sequential',
 'sometimes',
 'important',
 'things',
 'are',
 'happening',
 'simultaneously',
 'thousand',
 'leagues',
 'apart']